# 실행 순서
- 셀1~3까지는 그대로
- 셀4-1
- 셀 5-2
- 셀6-3   
### 여기서 다른 사용자를 랜덤으로 바꾸려먼 셀 4-1-a로 돌아가기, 이후부터는 셀5-2부터 그대로
- 셀7로 json 저장

# 셀 1) 패키지 설치

In [1]:
!pip install --quiet openai pymysql python-dotenv pandas

# 셀 2) 환경 변수 로드 (.env 사용 권장)

In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

# 키 점검
need = ["OPENAI_API_KEY","DB_HOST","DB_PORT","DB_USER","DB_PASS","DB_NAME"]
for k in need:
    assert os.getenv(k), f"{k} 없음"
print("✅ 환경 변수 OK")

✅ 환경 변수 OK


# 셀 3) DB 연결 + 테이블 구조 확인(중요)

In [23]:
import pymysql, pandas as pd

def get_conn():
    return pymysql.connect(
        host=os.getenv("DB_HOST"),
        port=int(os.getenv("DB_PORT")),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASS"),
        database=os.getenv("DB_NAME"),
        charset=os.getenv("DB_CHARSET","utf8mb4"),
        cursorclass=pymysql.cursors.DictCursor
    )

with get_conn() as conn:
    print("✅ DB 연결 성공")
    print("user_info_dummy_data 컬럼")
    print(pd.read_sql("DESCRIBE user_info_dummy_data;", conn))
    print("\nfridge_items_dummy_data 컬럼")
    print(pd.read_sql("DESCRIBE fridge_items_dummy_data;", conn))
    print("\nrecipe 컬럼(일부)")
    df = pd.read_sql("DESCRIBE recipe;", conn)
    display(df.head(12))

✅ DB 연결 성공
user_info_dummy_data 컬럼
   Field  Type  Null  Key  Default  Extra
0  Field  Type  Null  Key  Default  Extra
1  Field  Type  Null  Key  Default  Extra
2  Field  Type  Null  Key  Default  Extra
3  Field  Type  Null  Key  Default  Extra
4  Field  Type  Null  Key  Default  Extra
5  Field  Type  Null  Key  Default  Extra
6  Field  Type  Null  Key  Default  Extra
7  Field  Type  Null  Key  Default  Extra

fridge_items_dummy_data 컬럼
   Field  Type  Null  Key  Default  Extra
0  Field  Type  Null  Key  Default  Extra
1  Field  Type  Null  Key  Default  Extra
2  Field  Type  Null  Key  Default  Extra
3  Field  Type  Null  Key  Default  Extra

recipe 컬럼(일부)


C:\Users\Admin\AppData\Local\Temp\ipykernel_3764\407010714.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql("DESCRIBE user_info_dummy_data;", conn))
C:\Users\Admin\AppData\Local\Temp\ipykernel_3764\407010714.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql("DESCRIBE fridge_items_dummy_data;", conn))
C:\Users\Admin\AppData\Local\Temp\ipykernel_3764\407010714.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("DESCRIBE recipe;", conn)


DatabaseError: Execution failed on sql 'DESCRIBE recipe;': (1146, "Table 'lgup3.recipe' doesn't exist")

# 셀 4) 유저 프로필/냉장고 불러오기

## 셀4-1) 커서 기반으로 완전 교체   (pandas.read_sql 자꾸 에러나기 때문)

In [9]:
import pandas as pd

# 4-0) 공통 UserID 1명 뽑기 (JOIN으로 보장)
def pick_user_with_fridge() -> str:
    sql = """
    SELECT u.UserID AS uid
    FROM user_info_dummy_data u
    INNER JOIN fridge_items_dummy_data f
      ON f.UserID = u.UserID
    LIMIT 1;
    """
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            row = cur.fetchone()
            if not row:
                raise RuntimeError("공통 UserID가 없습니다. 두 테이블의 UserID 매칭을 확인하세요.")
            return row["uid"]

sample_user = pick_user_with_fridge()
print("샘플 UserID =", sample_user)

# 4-1) 프로필 조회 (cursor 기반)
def get_user_profile(user_id: str) -> dict:
    sql = """
    SELECT
      UserID        AS user_id,
      NAME          AS name,
      GENDER        AS gender,
      EMAIL         AS email,
      ID            AS login_id,
      PASSWORD      AS password,
      GOAL          AS goal_per_week,
      COOKING_LEVEL AS cooking_level
    FROM user_info_dummy_data
    WHERE UserID = %s
    """
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (user_id,))
            row = cur.fetchone()
            if not row:
                raise RuntimeError(f"user_info_dummy_data에서 프로필 없음: {user_id}")
            return row

# 4-2) 냉장고 재료 조회 (cursor 기반, 한글 컬럼만 백틱)
def get_user_fridge_items(user_id: str, limit: int = 200) -> pd.DataFrame:
    sql = f"""
    SELECT
      UserID   AS user_id,
      `재료`     AS item_name,
      `용량`     AS amount,
      `저장일시`  AS saved_at
    FROM fridge_items_dummy_data
    WHERE UserID = %s
    ORDER BY `저장일시` DESC
    LIMIT {int(limit)}
    """
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (user_id,))
            rows = cur.fetchall()
            if not rows:
                raise RuntimeError(f"fridge_items_dummy_data에서 냉장고 재료 없음: {user_id}")
            return pd.DataFrame(rows)

# --- 실행 & 확인 ---
profile = get_user_profile(sample_user)
fridge_df = get_user_fridge_items(sample_user)

print("프로필 확인(딕셔너리) ->", profile)         # 실제 값이 찍혀야 정상
display(pd.DataFrame([profile]))                   # 표로도 확인
display(fridge_df.head(10))


샘플 UserID = 249a8f76-ce74-41e3-8889-df58707d85d4
프로필 확인(딕셔너리) -> {'user_id': '249a8f76-ce74-41e3-8889-df58707d85d4', 'name': '이서윤', 'gender': 'female', 'email': 'jbvhea7d@example.com', 'login_id': 'jbvhea7d', 'password': 'FwWroy9T@*', 'goal_per_week': 5, 'cooking_level': '하'}


,user_id,name,gender,email,login_id,password,goal_per_week,cooking_level
0,249a8f76-ce74-41e3-8889-df58707d85d4,이서윤,female,jbvhea7d@example.com,jbvhea7d,FwWroy9T@*,5,하


,user_id,item_name,amount,saved_at
0,249a8f76-ce74-41e3-8889-df58707d85d4,두부(g),498,2025-10-13 12:18:59
1,249a8f76-ce74-41e3-8889-df58707d85d4,요거트(ml),111,2025-10-11 08:58:34
2,249a8f76-ce74-41e3-8889-df58707d85d4,사과(개),3,2025-10-05 08:46:44
3,249a8f76-ce74-41e3-8889-df58707d85d4,감자(g),454,2025-10-01 13:43:59
4,249a8f76-ce74-41e3-8889-df58707d85d4,굴(g),793,2025-09-24 18:41:50


## 셀 4-1-a) 사용자 랜덤으로 바꿔가며 테스트하기

In [18]:
def pick_random_user_with_fridge() -> str:
    sql = """
    SELECT u.UserID AS uid
    FROM user_info_dummy_data u
    JOIN fridge_items_dummy_data f ON f.UserID = u.UserID
    GROUP BY u.UserID
    ORDER BY RAND()
    LIMIT 1;
    """
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            row = cur.fetchone()
            assert row, "공통 UserID가 없습니다."
            return row["uid"]

# 랜덤 유저 뽑기
sample_user = pick_random_user_with_fridge()
print("샘플 UserID =", sample_user)

# 이어서 프로필/냉장고 읽기 → 셀4의 나머지 코드 동일
profile  = get_user_profile(sample_user)
fridge_df = get_user_fridge_items(sample_user)
display(pd.DataFrame([profile])); display(fridge_df.head(10))


샘플 UserID = 3a1264c8-cee4-4fa9-b8c3-b48fb051874e


,user_id,name,gender,email,login_id,password,goal_per_week,cooking_level
0,3a1264c8-cee4-4fa9-b8c3-b48fb051874e,차아름,female,jcep5qf4@example.com,jcep5qf4,ck1gRE3UEi,6,상


,user_id,item_name,amount,saved_at
0,3a1264c8-cee4-4fa9-b8c3-b48fb051874e,홍합(g),1072,2025-10-22 03:02:11
1,3a1264c8-cee4-4fa9-b8c3-b48fb051874e,포도(개),4,2025-10-20 22:18:51
2,3a1264c8-cee4-4fa9-b8c3-b48fb051874e,대파(g),148,2025-09-25 15:05:21


# 셀 5-1) 냉장고 재료 → 레시피 후보 뽑기 (AND 기본, 0건이면 OR로 폴백)

In [ ]:
import re
import pandas as pd

def normalize_ingredient(s: str) -> str:
    """괄호 안/공백 제거 등 아주 가벼운 정규화 (MVP)"""
    s = str(s)
    s = re.sub(r"\(.*?\)", "", s)   # "사과(개)" -> "사과"
    s = s.replace("/", " ")         # "돼지고기/목살" -> "돼지고기 목살"
    return s.strip()

def pick_keywords_from_fridge(fridge_df: pd.DataFrame, k: int = 4) -> list[str]:
    """냉장고 재료에서 상위 k개 키워드 추출(너무 많으면 검색이 과하게 좁아짐)"""
    return (
        fridge_df["item_name"]
        .map(normalize_ingredient)
        .dropna()
        .astype(str)
        .str.strip()
        .head(k)
        .tolist()
    )

def fetch_candidates_like(keywords: list[str], limit: int = 15, use_or: bool = False) -> list[dict]:
    """
    recipe.INGREDIENT_FULL 에 대해 LIKE 검색.
    - 기본: 모두 포함(AND)
    - 0건이면 OR 전략으로 폴백 권장
    """
    if not keywords:
        return []
    conj = " OR " if use_or else " AND "
    where = conj.join(["INGREDIENT_FULL LIKE %s"] * len(keywords))
    params = [f"%{kw}%" for kw in keywords]
    sql = f"""
    SELECT
      RECIPE_ID,
      RECIPE_NM_KO    AS title,
      COOKING_TIME    AS cook_time,
      `1QNT_CALORIE`  AS kcal,           -- 숫자 시작 컬럼은 백틱 사용
      INGREDIENT_FULL AS ingredients_text,
      STEP_TEXT       AS steps_text
    FROM recipe
    WHERE {where}
    LIMIT {int(limit)}
    """
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params)
            return cur.fetchall()  # list[dict]

# 1) 냉장고 → 키워드 뽑기
user_keywords = pick_keywords_from_fridge(fridge_df, k=4)
print("검색 키워드 =", user_keywords)

# 2) 후보 뽑기 (AND) → 0건이면 OR로 재시도
candidates = fetch_candidates_like(user_keywords, limit=15, use_or=False)
if not candidates:
    print("AND로 0건 → OR로 다시 시도합니다.")
    candidates = fetch_candidates_like(user_keywords, limit=15, use_or=True)

print(f"후보 개수 = {len(candidates)}")
pd.DataFrame(candidates)[["title","cook_time","kcal"]].head(10)

검색 키워드 = ['두부', '요거트', '사과', '감자']
AND로 0건 → OR로 다시 시도합니다.
후보 개수 = 15


,title,cook_time,kcal
0,카레라이스,30.0,162.5
1,감자수제비,60.0,102.5
2,만둣국,40.0,135.0
3,두부국,40.0,30.0
4,두부조개탕,30.0,32.5
5,구운감자와도미구이,60.0,112.5
6,두부드레싱과 채소샐러드,30.0,45.5
7,바질토마토두부샐러드,20.0,73.5
8,갈치무조림,40.0,55.0
9,두부다시마말이,30.0,55.0


# 셀5-2) 셀5-1를 (전체 교체): 모든 재료 사용 + 최근 재료 목록도 전달 + 다양성 유지

In [19]:
import re
import pandas as pd

def normalize_ingredient(s: str) -> str:
    s = str(s)
    s = re.sub(r"\(.*?\)", "", s)   # "사과(개)" → "사과"
    s = s.replace("/", " ")
    return s.strip()

def pick_keywords_from_fridge_all(fridge_df: pd.DataFrame, max_n: int = 30) -> list[str]:
    """냉장고 재료 전부(중복 제거) 사용. 너무 길어지지 않게 최대 max_n까지."""
    keys = (
        fridge_df["item_name"]
        .map(normalize_ingredient)
        .dropna().astype(str).str.strip()
        .drop_duplicates()
        .tolist()
    )
    return keys[:max_n]

def recent_items_from_fridge(fridge_df: pd.DataFrame, m: int = 8) -> list[str]:
    """저장일시 최신 순 상위 m개 재료 이름(정규화 후)"""
    tmp = fridge_df.copy()
    tmp["saved_at"] = pd.to_datetime(tmp["saved_at"], errors="coerce")
    tmp = tmp.sort_values("saved_at", ascending=False)
    return (
        tmp["item_name"]
        .map(normalize_ingredient)
        .dropna().astype(str).str.strip()
        .head(m).tolist()
    )

def fetch_candidates_like(keywords: list[str], limit: int = 40, use_or: bool = False) -> list[dict]:
    if not keywords:
        return []
    conj = " OR " if use_or else " AND "
    where = conj.join(["INGREDIENT_FULL LIKE %s"] * len(keywords))
    params = [f"%{kw}%" for kw in keywords]
    sql = f"""
    SELECT
      RECIPE_ID,
      RECIPE_NM_KO    AS title,
      COOKING_TIME    AS cook_time,
      `1QNT_CALORIE`  AS kcal,
      INGREDIENT_FULL AS ingredients_text,
      STEP_TEXT       AS steps_text
    FROM recipe
    WHERE {where}
    LIMIT {int(limit)}
    """
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params)
            return cur.fetchall()

def guess_main_ingredient(c: dict) -> str:
    txt = (c.get("ingredients_text") or "").split(",")[0]
    if not txt:
        txt = (c.get("title") or "").split()[0]
    txt = re.sub(r"\(.*?\)", "", txt).strip()
    return txt

def diversify_candidates(candidates: list[dict], want: int = 12, max_per_main: int = 1) -> list[dict]:
    buckets = {}
    for c in candidates:
        main = guess_main_ingredient(c) or "기타"
        buckets.setdefault(main, [])
        if len(buckets[main]) < max_per_main:
            buckets[main].append(c)
    diverse = []
    for arr in buckets.values():
        diverse.extend(arr)
        if len(diverse) >= want:
            break
    return diverse[:want]

# --- 실행 흐름 ---
user_keywords = pick_keywords_from_fridge_all(fridge_df, max_n=30)   # ← 냉장고 재료 전부(중복 제거)
recent_emphasis = recent_items_from_fridge(fridge_df, m=8)           # ← 최신 재료 목록
print("검색 키워드 =", user_keywords[:10], "... 총", len(user_keywords), "개")  # 너무 길면 일부만 미리보기
print("최근 재료(강조) =", recent_emphasis)

candidates = fetch_candidates_like(user_keywords, limit=60, use_or=False)
if not candidates:
    print("AND로 0건 → OR로 재시도")
    candidates = fetch_candidates_like(user_keywords, limit=60, use_or=True)

print("원본 후보 수:", len(candidates))
candidates = diversify_candidates(candidates, want=12, max_per_main=1)  # 주재료 다양성 보장
print("다양성 적용 후보 수:", len(candidates))

pd.DataFrame(candidates)[["title","cook_time","kcal"]].head(10)


검색 키워드 = ['홍합', '포도', '대파'] ... 총 3 개
최근 재료(강조) = ['홍합', '포도', '대파']
AND로 0건 → OR로 재시도
원본 후보 수: 60
다양성 적용 후보 수: 12


,title,cook_time,kcal
0,열무김치냉면,25.0,156.25
1,해물국수,40.0,132.50
2,미역냉국,30.0,20.50
3,오이냉국,20.0,20.50
4,해산물샐러드,30.0,90.00
5,죽순표고버섯볶음나물,20.0,26.25
6,부추표고버섯볶음,30.0,37.50
7,바질토마토두부샐러드,20.0,73.50
8,갈치무조림,40.0,55.00
9,닭불고기,35.0,90.00


# 셀 6-1) LLM 호출 → 상위 3개 카드(JSON만 반환)

In [11]:
import json, re
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

SYSTEM = (
  "너는 한국어 요리 추천 도우미야. 반드시 '순수 JSON 객체'만 반환해."
  '출력 스키마는 {"items":[{"title":string,"match_score":integer,"cook_time":string,"kcal":string,"why":string,"top_ingredients":[string]}]} 하나만 사용.'
)

def _extract_json(text: str) -> str:
    t = re.sub(r"^```(?:json)?", "", text.strip())
    t = re.sub(r"```$", "", t)
    m = re.search(r"\{.*\}", t, re.S)
    return m.group(0) if m else t

def recommend_with_llm(profile: dict, fridge_df: pd.DataFrame, candidates: list[dict]) -> dict:
    """후보를 LLM에 넘겨 상위 3개만 JSON으로 받기"""
    # 사용자 요약(프롬프트에 쓰일 부분)
    user_summary = {
        "name": profile.get("name"),
        "gender": profile.get("gender"),
        "goal_per_week": profile.get("goal_per_week"),
        "cooking_level": profile.get("cooking_level"),
        "fridge_items_sample": fridge_df["item_name"].map(str).head(10).tolist()
    }

    user_msg = f"""
[사용자]
- 이름: {user_summary['name']}, 성별: {user_summary['gender']}
- 주간 요리 목표: {user_summary['goal_per_week']}회
- 요리 레벨: {user_summary['cooking_level']} (상=어려운 요리 가능 / 하=쉬운 요리 우대)
- 냉장고 재료(일부): {user_summary['fridge_items_sample']}

[요청]
아래 candidates(레시피 후보) 중에서
1) 냉장고 재료와의 적합도, 2) 레벨이 '하'면 쉬운 요리 가산점, 3) 조리시간 짧을수록 가산점을 기준으로
**상위 3개**만 골라줘.

[엄격 규칙]
- cook_time, kcal은 후보에 값이 있을 때만 '그대로 복사', 없으면 "".
- 임의로 수치 생성 금지.
- ingredients_text에서 핵심 재료명만 최대 6개 추출하여 top_ingredients에 넣기.
- 결과는 오직 JSON 객체 하나만.

[candidates]
{json.dumps(candidates, ensure_ascii=False)}
"""
    # JSON 강제 모드 우선
    try:
        rsp = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0.1,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": SYSTEM},
                {"role": "user", "content": user_msg},
            ],
        )
        return json.loads(rsp.choices[0].message.content)
    except Exception:
        # 백업: 코드블록 제거 후 파싱
        rsp = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0.1,
            messages=[
                {"role": "system", "content": SYSTEM},
                {"role": "user", "content": user_msg},
            ],
        )
        cleaned = _extract_json(rsp.choices[0].message.content)
        return json.loads(cleaned)

# 실행
if not candidates:
    raise RuntimeError("레시피 후보가 없습니다. 키워드 수를 늘리거나 OR 검색을 사용하세요.")

result = recommend_with_llm(profile, fridge_df, candidates)
result  # 노트북에서 JSON 구조 바로 확인

{'items': [{'title': '바질토마토두부샐러드',
   'match_score': 10,
   'cook_time': '20.0',
   'kcal': '73.5',
   'why': '두부를 사용하여 냉장고 재료와 잘 어울리며, 조리 시간이 짧고 쉬운 요리입니다.',
   'top_ingredients': ['두부', '토마토', '바질', '다진양파', '식초', '간장']},
  {'title': '두부국',
   'match_score': 9,
   'cook_time': '40.0',
   'kcal': '30.0',
   'why': '두부를 사용하여 냉장고 재료와 잘 어울리며, 쉬운 요리입니다.',
   'top_ingredients': ['두부', '쇠고기', '파', '다진마늘', '고추장', '참기름']},
  {'title': '두부드레싱과 채소샐러드',
   'match_score': 8,
   'cook_time': '30.0',
   'kcal': '45.5',
   'why': '두부를 사용하여 냉장고 재료와 잘 어울리며, 조리 시간이 짧고 쉬운 요리입니다.',
   'top_ingredients': ['상추잎', '두부', '깨소금', '꿀', '레몬즙', '당근']}]}

# 셀6-2) 셀6-1을 “텍스트로 보기 쉬운 출력”으로 변경

In [12]:
# === 셀6: LLM을 텍스트 응답으로 사용 ===
import json, re
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def recommend_text_style(profile: dict, fridge_df: pd.DataFrame, candidates: list[dict]) -> str:
    # 1) 사용자 요약 만들기
    user_summary = {
        "name": profile.get("name"),
        "gender": profile.get("gender"),
        "goal_per_week": profile.get("goal_per_week"),
        "cooking_level": profile.get("cooking_level"),
        "fridge_items_sample": fridge_df["item_name"].map(str).head(10).tolist()
    }

    # 2) 프롬프트(부탁글) 만들기 — 텍스트로 ‘사람이 읽기 좋게’ 답해달라고 요청
    user_msg = f"""
[사용자]
- 이름: {user_summary['name']}, 성별: {user_summary['gender']}
- 주간 요리 목표: {user_summary['goal_per_week']}회
- 요리 레벨: {user_summary['cooking_level']} (상=어려운 요리 가능 / 하=쉬운 요리 우대)
- 냉장고 재료(일부): {user_summary['fridge_items_sample']}

[후보 레시피 목록(원본 데이터)]
다음은 DB에서 찾은 레시피 후보들이야. 각 항목에는 제목(title), 조리시간(cook_time), 칼로리(kcal), 재료문장(ingredients_text)이 들어 있어.
{json.dumps(candidates, ensure_ascii=False)}

[요청]
- 위 후보 중에서 다음 기준으로 **상위 3개**를 골라줘.
  1) 냉장고 재료와의 적합도(겹치는 재료가 많을수록 좋음)
  2) 사용자의 요리 레벨과의 적합
  3) 냉장고 재료 저장일시가 최신인 것들 위주
- **사람이 읽기 편한 텍스트**로, 아래 양식처럼 써줘.

[출력 양식 예시]
1) 레시피명 — 예상 난이도/시간: (예: 쉬움, 20분) / 칼로리: (있으면 숫자, 없으면 미기재)
   - 이유: (왜 골랐는지 간단히)
   - 핵심 재료: (최대 6개 키워드)

2) ...
3) ...
"""

    rsp = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.3,  # 텍스트라 약간 창의성 허용
        messages=[
            {"role": "system", "content": "너는 한국어 요리 추천 도우미야. 사용자에게 읽기 쉬운 문장으로 간결하게 답해."},
            {"role": "user", "content": user_msg}
        ],
    )
    return rsp.choices[0].message.content

# 실행
if not candidates:
    raise RuntimeError("레시피 후보가 없습니다. 셀5에서 AND→OR 폴백을 확인하세요.")

text_result = recommend_text_style(profile, fridge_df, candidates)
print(text_result)  # ← 텍스트로 바로 읽기 좋게 출력


1) 두부국 — 예상 난이도/시간: 쉬움, 40분 / 칼로리: 30
   - 이유: 두부가 냉장고 재료에 있어 적합하며, 요리 난이도가 낮고 조리 시간이 적당합니다.
   - 핵심 재료: 두부, 쇠고기, 파, 다진마늘, 고추장, 소금

2) 두부드레싱과 채소샐러드 — 예상 난이도/시간: 쉬움, 30분 / 칼로리: 45.5
   - 이유: 두부와 다양한 채소를 활용하여 건강한 샐러드를 만들 수 있으며, 요리 난이도가 낮습니다.
   - 핵심 재료: 두부, 상추, 당근, 파프리카, 레몬즙, 꿀

3) 바질토마토두부샐러드 — 예상 난이도/시간: 쉬움, 20분 / 칼로리: 73.5
   - 이유: 두부와 신선한 재료를 사용하여 간단하게 만들 수 있는 요리로, 요리 난이도가 낮습니다.
   - 핵심 재료: 두부, 토마토, 바질, 다진양파, 식초, 간장


# 셀6-3) 셀6-2을 교체: 읽기 쉬운 “요약 헤더 + 서로 다른 주재료 3개” 텍스트 출력

In [ ]:
# === 셀6: 텍스트 응답 (적합도/레벨/최근/주재료 불중복 + 원문 재료/조리 포함) ===
import json, re
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def recommend_text_style(profile: dict,
                         fridge_df: pd.DataFrame,
                         candidates: list[dict],
                         recent_emphasis: list[str]) -> str:
    name = profile.get("name") or "사용자"
    level = profile.get("cooking_level") or "-"
    fridge_list = ", ".join(fridge_df["item_name"].map(str).head(12).tolist())

    user_msg = f"""
[요약]
- {name}님의 냉장고 재료: {fridge_list}
- 최근에 저장한 재료(신선도 우선 고려): {recent_emphasis}

[목표]
- 아래 후보 레시피 중에서 **3가지를 추천**해주세요.
- 기준:
  1) **사용자 냉장고 재료와의 적합도**(겹치는 재료가 많을수록 좋음)
  2) **사용자의 요리 레벨 '{level}'에 맞게** (레벨이 '하'면 쉬운 요리 위주)
  3) **냉장고 재료 저장일시가 최신인 재료**를 활용한 요리 우선
  4) **세 레시피의 주재료는 서로 겹치지 않게**

[후보(원본 데이터)]
- 각 후보는 title, cook_time, kcal, ingredients_text(원문 재료), steps_text(원문 조리)가 있습니다.
- 없는 값은 비워두고 **새로 만들어 넣지 마세요**(추측 금지).
{json.dumps(candidates, ensure_ascii=False)}

[출력 형식(텍스트)]
"{name}님의 냉장고를 바탕으로 다음 3가지 레시피를 추천드려요 (주재료 중복 없음)."
1) 레시피명 — 난이도/시간: (예: 쉬움, 20분) / 칼로리: (있으면 수치)
   - 이유: (적합도·레벨·최근 재료 고려 근거)
   - 핵심 재료: (최대 6개)
   - 원문 재료(ingredients): (후보에 있을 경우 그대로)
   - 원문 조리(steps): (후보에 있을 경우 그대로)

2) ...
3) ...
"""

    rsp = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.3,
        messages=[
            {"role": "system", "content": "너는 한국어 요리 추천 도우미야. 친절하고 간결하게, 거짓 없이 원문을 그대로 사용해."},
            {"role": "user", "content": user_msg}
        ],
    )
    return rsp.choices[0].message.content

# 실행
if not candidates:
    raise RuntimeError("레시피 후보가 없습니다. 셀5에서 OR 폴백을 확인하세요.")

text_result = recommend_text_style(profile, fridge_df, candidates, recent_emphasis)
print(text_result)


"차아름님의 냉장고를 바탕으로 다음 3가지 레시피를 추천드려요 (주재료 중복 없음)."

1) 해물국수 — 상/40분 / 칼로리: 132.5
   - 이유: 홍합과 대파가 포함되어 있어 적합하며, 요리 난이도도 상급에 맞습니다.
   - 핵심 재료:
     • 국수
     • 돼지고기
     • 오징어
     • 새우
     • 표고버섯
   - 원문 재료(ingredients):
     • 국수 (400g)
     • 돼지고기 (100g)
     • 녹말 (1큰술)
     • 계란흰자 (1개)
     • 오징어 (1/2마리)
     • 새우 (100g)
     • 홍합 (100g)
     • 표고버섯 (2개)
     • 죽순 (2개)
     • 파 (약간)
     • 다진마늘 (약간)
     • 멸칫국물 (8컵)
     • 청주 (약간)
     • 소금 (약간)
   - 원문 조리(steps):
     1. 고기는 저며서 녹말가루, 계란흰자, 청주, 소금으로 양념한다.
     2. 해물은 한 입 크기로 손질해 데친다.
     3. 채소는 얄팍하게 저며썬다.
     4. 팬에 기름을 둘러 다진 마늘을 볶다가 고기를 넣어 볶는다.
     5. 어느정도 익으면 멸치국물을 부어 끓이다가 해물과 채소를 넣고 간을 한다.
     6. 삶은 국수에 ⑤의 해물장국을 부어낸다.

2) 미역냉국 — 중/30분 / 칼로리: 20.5
   - 이유: 대파가 포함되어 있으며, 요리 난이도도 중급에 적합합니다.
   - 핵심 재료:
     • 불린미역
     • 오이
     • 간장
   - 원문 재료(ingredients):
     • 불린미역 (2컵)
     • 오이 (1/2개)
     • 대파 (1/2뿌리)
     • 다진마늘 (1/2작은술)
     • 깨소금 (약간)
     • 소금 (약간)
     • 고춧가루 (약간)
     • 간장 (2큰술)
     • 물 (10컵)
   - 원문 조리(steps):
   

# 셀 7) 보기 좋게 표로 확인 + 파일 저장

In [ ]:
items = result.get("items", [])
display(pd.DataFrame(items))

with open("recommend_result.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print("저장 완료: recommend_result.json")


# 셀8) 같은 추천을 “JSON 스키마”로 다시 받아서 CSV 저장

In [21]:
# === JSON 구조로도 한번 더 받아 CSV 저장 ===
import json, re
import pandas as pd
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def recommend_json_style(profile: dict,
                         fridge_df: pd.DataFrame,
                         candidates: list[dict],
                         recent_emphasis: list[str]) -> dict:
    """텍스트 대신 '구조화 JSON'으로 똑같은 추천 3개를 받아오기"""
    name = profile.get("name") or "사용자"
    level = profile.get("cooking_level") or "-"
    fridge_list = ", ".join(fridge_df["item_name"].map(str).head(12).tolist())

    # JSON 스키마 가이드 (프론트/CSV 변환용으로 안정적)
    SCHEMA_NOTE = """
반드시 아래 JSON 스키마만 반환하세요(불필요한 텍스트/코드블럭 금지):
{
  "items": [
    {
      "title": "string",
      "difficulty": "string",         // 예: "쉬움", "어려움" (없으면 "")
      "time": "string",               // 예: "20분", "40분" (없으면 "")
      "reason": "string",             // 추천 이유 (겹치지 않게 작성)
      "key_ingredients": ["string"],  // 최대 6개
      "ingredients_text": "string",   // 원문 재료(있는 경우 그대로)
      "steps_text": "string"          // 원문 조리(있는 경우 그대로, 줄바꿈 포함 가능)
    }
  ]
}
"""

    user_msg = f"""
[요약]
- {name}님의 냉장고 재료: {fridge_list}
- 최근에 저장한 재료(신선도 우선 고려): {recent_emphasis}

[목표]
- 아래 후보 레시피 중에서 **3가지를 추천**해주세요.
- 기준:
  1) **사용자 냉장고 재료와의 적합도**(겹치는 재료가 많을수록 좋음)
  2) **사용자의 요리 레벨 '{level}'에 맞게** (레벨이 '하'면 쉬운 요리 위주)
  3) **냉장고 재료 저장일시가 최신인 재료**를 활용한 요리 우선
  4) **세 레시피의 주재료는 서로 겹치지 않게**

[후보(원본 데이터)]
{json.dumps(candidates, ensure_ascii=False)}

[요청]
- {SCHEMA_NOTE}
"""

    rsp = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.2,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "너는 한국어 요리 추천 도우미야. 거짓 없이 원문을 그대로 사용하고, 지정한 JSON 스키마만 반환해."},
            {"role": "user", "content": user_msg}
        ],
    )
    return json.loads(rsp.choices[0].message.content)

# 실행: JSON으로 받아 CSV 저장
rec_json = recommend_json_style(profile, fridge_df, candidates, recent_emphasis)
df = pd.DataFrame(rec_json.get("items", []))
display(df)

# 엑셀에서 한글이 깨지지 않게 'utf-8-sig'로 저장 추천
df.to_csv("recipe_llm_test.csv", index=False, encoding="utf-8-sig")
print("저장 완료: recipe_llm_test.csv")


,title,difficulty,time,reason,key_ingredients,ingredients_text,steps_text
0,해물국수,어려움,40분,"사용자 재료인 홍합을 활용하며, 요리 레벨에 맞는 복잡한 조리법이 포함되어 있습니다.","[국수, 돼지고기, 오징어, 새우, 홍합, 표고버섯]","['국수 (400g)', '돼지고기 (100g)', '녹말 (1큰술)', '계란흰자...","['1. 고기는 저며서 녹말가루, 계란흰자, 청주, 소금으로 양념한다.', '2. ..."
1,미역냉국,어려움,30분,"사용자 재료인 대파를 활용하며, 요리 레벨에 맞는 조리법입니다.","[미역, 오이, 대파]","['불린미역 (2컵)', '오이 (1/2개)', '대파 (1/2뿌리)', '다진마늘...","['1. 미역은 줄기를 떼어 티를 골라낸 다음 찬물에 담가30분 정도 불린다.', ..."
2,죽순표고버섯볶음나물,어려움,20분,"사용자 재료인 대파를 활용하며, 다른 재료와 겹치지 않는 요리입니다.","[죽순, 표고버섯, 대파]","['죽순 (2개)', '표고버섯 (8장)', '대파 (1뿌리)', '식용유 (2큰술...","['1. 죽순은 빗살모양을 살려 얇게 저며 썬다.', '2. 불린 표고는 기둥을 뗀..."


저장 완료: recipe_llm_test.csv
